In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    GPT2Tokenizer, GPT2LMHeadModel
)
from transformers.generation.streamers import TextStreamer

# =============================
# 1. Load Binary Classification Model (OOD: Yes/No)
# =============================
classification_model_path = '/content/drive/MyDrive/Chatbot_Query_Classifier/chatbot_query_classifier_distilbert_model'
clf_tokenizer = AutoTokenizer.from_pretrained(classification_model_path)
clf_model = AutoModelForSequenceClassification.from_pretrained(classification_model_path)
clf_model.eval()
clf_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clf_model.to(clf_device)

# =============================
# 2. Load GPT-2 Chatbot Model
# =============================
gpt2_model_path = '/content/drive/MyDrive/Chatbot/Final_Advanced_event_ticketing_DistilGPT2_fine-tuned'
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model_path)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_path)
gpt2_model.eval()
gpt2_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2_model.to(gpt2_device)

# =============================
# 3. Random OOD Fallback Responses
# =============================
fallback_responses = [
    "I’m sorry, but I am unable to assist with this request. If you need help regarding event tickets, I’d be happy to support you.",
    "Apologies, but I am not able to provide assistance on this matter. Please let me know if you require help with event tickets.",
    "Unfortunately, I cannot assist with this. However, I am here to help with any event ticket-related concerns you may have.",
    "Regrettably, I am unable to assist with this request. If there's anything I can do regarding event tickets, feel free to ask.",
    "I regret that I am unable to assist in this case. Please reach out if you need support related to event tickets.",
    "Apologies, but this falls outside the scope of my support. I’m here if you need any help with event ticket issues.",
    "I'm sorry, but I cannot assist with this particular topic. If you have questions about event tickets, I’d be glad to help.",
    "I regret that I’m unable to provide assistance here. Please let me know how I can support you with event ticket matters.",
    "Unfortunately, I am not equipped to assist with this. If you need help with event tickets, I am here for that.",
    "I apologize, but I cannot help with this request. However, I’d be happy to assist with anything related to event tickets.",
    "I’m sorry, but I’m unable to support this request. If it’s about event tickets, I’ll gladly help however I can.",
    "This matter falls outside the assistance I can offer. Please let me know if you need help with event ticket-related inquiries.",
    "Regrettably, this is not something I can assist with. I’m happy to help with any event ticket questions you may have.",
    "I’m unable to provide support for this issue. However, I can assist with concerns regarding event tickets.",
    "I apologize, but I cannot help with this matter. If your inquiry is related to event tickets, I’d be more than happy to assist.",
    "I regret that I am unable to offer help in this case. I am, however, available for any event ticket-related questions.",
    "Unfortunately, I’m not able to assist with this. Please let me know if there’s anything I can do regarding event tickets.",
    "I'm sorry, but I cannot assist with this topic. However, I’m here to help with any event ticket concerns you may have.",
    "Apologies, but this request falls outside of my support scope. If you need help with event tickets, I’m happy to assist.",
    "I’m afraid I can’t help with this matter. If there’s anything related to event tickets you need, feel free to reach out.",
    "This is beyond what I can assist with at the moment. Let me know if there’s anything I can do to help with event tickets.",
    "Sorry, I’m unable to provide support on this issue. However, I’d be glad to assist with event ticket-related topics.",
    "Apologies, but I can’t assist with this. Please let me know if you have any event ticket inquiries I can help with.",
    "I’m unable to help with this matter. However, if you need assistance with event tickets, I’m here for you.",
    "Unfortunately, I can’t support this request. I’d be happy to assist with anything related to event tickets instead.",
    "I’m sorry, but I can’t help with this. If your concern is related to event tickets, I’ll do my best to assist.",
    "Apologies, but this issue is outside of my capabilities. However, I’m available to help with event ticket-related requests.",
    "I regret that I cannot assist with this particular matter. Please let me know how I can support you regarding event tickets.",
    "I’m sorry, but I’m not able to help in this instance. I am, however, ready to assist with any questions about event tickets.",
    "Unfortunately, I’m unable to help with this topic. Let me know if there's anything event ticket-related I can support you with."
]

# =============================
# 4. Optional Streamer to Trim Whitespace
# =============================
class TrimLeadingTextStreamer(TextStreamer):
    def __init__(self, tokenizer, skip_prompt=True, **kwargs):
        try:
            super().__init__(tokenizer, skip_prompt=skip_prompt, **kwargs)
        except TypeError:
            decode_kwargs = kwargs.pop("decode_kwargs", {})
            decode_kwargs.setdefault("skip_special_tokens", True)
            super().__init__(tokenizer, skip_prompt=skip_prompt, decode_kwargs=decode_kwargs)
        self._started = False

    def on_finalized_text(self, text, stream_end=False):
        if not self._started:
            text = text.lstrip()
            if text:
                self._started = True
            else:
                return
        return super().on_finalized_text(text, stream_end=stream_end)

# =============================
# 5. Classify Whether Input is OOD
# =============================
def is_ood(query: str) -> bool:
    inputs = clf_tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(clf_device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = clf_model(**inputs)
    pred_id = torch.argmax(outputs.logits, dim=1).item()
    return pred_id == 1  # True if OOD

# =============================
# 6. Generate Response with GPT-2 (LIVE Streaming!)
# =============================
def generate_response_live(instruction: str, max_length=256):
    prompt = f"Instruction: {instruction} Response:"
    inputs = gpt2_tokenizer(prompt, return_tensors="pt", padding=True).to(gpt2_device)

    print("🤖 Bot: ", end="", flush=True)  # Prefix bot response

    streamer = TrimLeadingTextStreamer(gpt2_tokenizer, skip_prompt=True, skip_special_tokens=True)

    with torch.no_grad():
        gpt2_model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.5,
            top_p=0.95,
            do_sample=True,
            pad_token_id=gpt2_tokenizer.eos_token_id,
            streamer=streamer  # 🔴 This will print live!
        )

In [ ]:
# =============================
# ❇️ Start Interaction
# =============================

print("\n🤖 Welcome to the Event Ticketing Assistant!\n")
user_input = input("🔹 You: ").strip()

# Exit early if input is empty
if not user_input:
    print("⚠️ Please enter a query.")
else:
    # Capitalize the first letter for better GPT-2 formatting
    formatted_input = user_input[0].upper() + user_input[1:] if user_input else ""

    if is_ood(user_input):
        bot_response = random.choice(fallback_responses)
        print(f"🤖 Bot: {bot_response}")
    else:
        generate_response_live(formatted_input)  # 🔴 Live response only, nothing printed afterwards


🤖 Welcome to the Event Ticketing Assistant!

🔹 You: How to sell my ticket
🤖 Bot: Beginning the process of selling or exchanging your event ticket is simple. Kindly adhere to the following instructions:

1. Go to {{WEBSITE_URL}} and enter your credentials to log in.
2. Proceed to the {{TICKET_SECTION}} area.
3. Identify and choose the ticket you wish to sell or exchange.
4. Click on the {{SELL_TICKET_OPTION}} button.
5. Fill in the necessary details and confirm your choice.

By adhering to these steps, you can efficiently handle your event tickets. If any issues arise, do not hesitate to seek further support.
